In [101]:
import requests
import pandas as pd
from tqdm import tqdm
S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

COMMON_PARAMS = {
    "action":"query",
    "format":"json",
    "list":"categorymembers",
    "cmlimit": 500
}

PAGES_LIST = []

PAGES_DICT = {
}

FIRST_RUN = True

SKIP_UNTIL = False

In [113]:
CATEGORIES_LIST = pd.read_csv("scraped_categories.csv")['0'].tolist()
CATEGORIES_LIST.append("Category:Archaeological artifacts")

In [93]:
def scrape_page_text(title):
    PARAMS = {
        "action":"query",
        "format":"json",
        'prop': 'extracts',
        'explaintext': True,
        'titles': title
    }
    R = S.get(url=URL, params=PARAMS)
    article = R.json()
    pages = article['query']['pages']
    page_id = next(iter(pages))
    text = pages[page_id]['extract']
    return text

In [94]:
scrape_page_text("Ain Sakhri Lovers")

'The Ain Sakhri Lovers figurine is a sculpture that was found in one of the Ain Sakhri caves near Bethlehem. The sculpture is considered to be 11,000 years old and to be the oldest known representation of two people engaged in sexual intercourse.\n\n\n== Discovery ==\nThe sculpture was identified in 1933 by René Neuville, a French consul in Jerusalem and prehistorian, when looking through random finds obtained by the French Fathers at Bethlehem. He found the stone whilst visiting a small museum with Abbé Breuil. Neuville immediately identified it as important and was able to get an introduction to the Bedouin who had made the finds at Wadi Khareitoun. He was led to a location within the Ain Sakhri caves and it is from these caves that the sculpture gets its name. Excavations of the caves revealed that the cave had been used domestically thousands of years ago and the finds were Natufian. For this reason it is thought that the figurine was used domestically and had not been left there a

In [95]:
def scrape_page_wikidata(title):
    PARAMS = {
        "prop": "pageprops",
        "ppprop": "wikibase_item",
        "format": "json",
        "action": "query",
        "titles": title
    }
    R = S.get(url=URL, params=PARAMS)
    metadata = R.json()
    metadata = metadata['query']['pages']
    page_id = next(iter(metadata))
    entity = metadata[page_id]["pageprops"]["wikibase_item"]
    return entity

In [96]:
scrape_page_wikidata("William the Faience Hippopotamus")

'Q137836'

In [97]:
def rec_scrape_pages(category_name, cmcontinue = False):
    PARAMS = COMMON_PARAMS
    PARAMS["cmtype"] = "page"
    PARAMS["cmtitle"] = category_name
    R = S.get(url=URL, params=PARAMS)
    articles = R.json()
    if articles.get('continue', False):
        cmcontinue = articles['continue']['cmcontinue']
    payload = articles['query']['categorymembers']
    for page in payload:
        title = page['title']
        if page in PAGES_LIST:
            continue
        else:
            try:
                text = scrape_page_text(title)
                entity = scrape_page_wikidata(title)
                PAGES_DICT[title] = {
                    "category": category_name,
                    "title": title,
                    "text": text,
                    "entity": entity
                }
            except Exception as e:
                print("An exception occurred at page ", title)
                print(e)
    if cmcontinue:
        rec_scrape_pages(category_name, cmcontinue)

In [110]:
for idx, category in tqdm(enumerate(CATEGORIES_LIST)):
    global FIRST_RUN
    global PAGES_DICT
    global PAGES_LIST
    global SKIP_UNTIL
    if category == SKIP_UNTIL:
        SKIP_UNTIL = False
    if SKIP_UNTIL:
        continue
    rec_scrape_pages(category)
    PAGES_LIST.extend(PAGES_DICT.keys())
    if FIRST_RUN:
        df = pd.DataFrame.from_dict(PAGES_DICT, orient="index").reset_index(drop=True)
        FIRST_RUN = False
    else:
        old = pd.read_csv("wiki_text.csv")
        new = pd.DataFrame.from_dict(PAGES_DICT, orient="index").reset_index(drop=True)
        df = pd.concat([old, new])
    if idx%10 == 0:
        df = df[df.columns.drop(list(df.filter(regex='Unnamed: 0')))]
        df.to_csv("wiki_text.csv")
        PAGES_DICT = {}

0it [00:00, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

1it [00:03,  3.38s/it]

An exception occurred at page  A brotherly quarrel
'pageprops'
An exception occurred at page  A Goddess Travels to Egypt
'pageprops'
An exception occurred at page  Alashiya-King, to Pharaoh: The Hand of (god)-Nergal
'pageprops'
An exception occurred at page  Assyria Joins the International Scene
'pageprops'
An exception occurred at page  Duty-Free
'pageprops'
An exception occurred at page  Love and Gold
'pageprops'
An exception occurred at page  The Missing Gold Statues Again
'pageprops'
An exception occurred at page  The Pharaoh's reproach Answered
'pageprops'
An exception occurred at page  Tushratta to Queen Mother Tiye, and missing gold statues
'pageprops'


2it [00:10,  4.42s/it]

An exception occurred at page  Papyrus Oxyrhynchus 1596
'pageprops'
An exception occurred at page  Papyrus Oxyrhynchus 1597
'pageprops'
An exception occurred at page  Papyrus Oxyrhynchus 1598
'pageprops'


3it [00:12,  3.63s/it]

...

40it [00:58,  2.19s/it]

An exception occurred at page  Artemision deposit
'pageprops'


41it [01:04,  3.40s/it]

An exception occurred at page  Ribchester Hoard
'pageprops'


44it [02:12, 19.80s/it]


In [114]:
cl = []
for category in CATEGORIES_LIST:
    if not category in categories:
        cl.append(category)
cl

['Category:Ancient Greek vases',
 'Category:Oxyrhynchus papyri vol. XIII',
 'Category:Oxyrhynchus papyri vol. XVIII',
 'Category:Oxyrhynchus papyri vol. XXIV',
 'Category:Oxyrhynchus papyri vol. XXXIV',
 'Category:Oxyrhynchus papyri vol. LXIV',
 'Category:Oxyrhynchus papyri vol. LXXI',
 'Category:Oxyrhynchus papyri vol. LXXII',
 'Category:Oxyrhynchus papyri vol. LXXIII',
 'Category:Oxyrhynchus papyri vol. LXXIV',
 'Category:High crosses by country',
 'Category:High crosses in Ireland',
 'Category:Lombard sculpture',
 'Category:Hill figures in the United Kingdom',
 'Category:Crosses in art',
 'Category:Stone crosses in Asia',
 'Category:Monumental crosses by country',
 'Category:Prehistoric art in Portugal',
 'Category:Rock art by continent',
 'Category:Great Basin rock art',
 'Category:Stone circles in Scotland',
 'Category:Obelisks by country',
 'Category:Obelisks in Italy',
 'Category:Obelisks in the United Kingdom',
 'Category:Archaeological artifacts by century',
 'Category:Archaeo

In [109]:
CATEGORIES_LIST = cl

In [90]:
scraped.to_csv("wiki_text.csv")